<a href="https://colab.research.google.com/github/SiracencoSerghei/DataScienceHW/blob/main/HW5/hw_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import iqr
from sklearn.impute import SimpleImputer

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

# # Розпаковка даних
# !unzip '/content/drive/My Drive/homework.zip' -d /content/homework
# base_path = '/content/homework/data'

In [2]:
base_path = './homework/data'

In [3]:
# Завантаження даних
def load_data(base_path):
    folder = Path(base_path)
    data_set = pd.DataFrame()
    for activity_folder in folder.iterdir():
        if activity_folder.is_dir():
            for file in activity_folder.iterdir():
                if file.suffix == '.csv':
                    df = pd.read_csv(file)
                    df['activity'] = activity_folder.name
                    data_set = pd.concat([data_set, df], ignore_index=True)
    return data_set

data_set = load_data(base_path)

In [4]:
data_set

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,-3.270479,5.147534,-3.198653,running
1,-0.210690,-9.184157,-1.288081,running
2,34.342430,28.064833,10.127473,running
3,4.951209,16.409859,-2.662352,running
4,2.973599,-7.393295,-2.920926,running
...,...,...,...,...
193855,8.288726,-10.783484,-3.165135,stairs
193856,-0.991199,-19.416977,4.074931,stairs
193857,7.062895,-8.063671,-0.972046,stairs
193858,0.766145,-4.788403,-0.981623,stairs


---
# 1. Навчання моделей без часових ознак
---
Спочатку навчимо моделі на сирих даних (без додаткових ознак).

In [5]:
# Підготовка даних
features = data_set.columns[:-1]  # Всі колонки, крім останньої ('activity')
X = data_set[features]
y = data_set['activity']

# Нормалізація даних
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Розділення на тренувальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.5, stratify=y, random_state=42)

In [6]:
# Навчання моделі SVM
svm_linear = SVC(kernel='linear', C = 1, random_state = 0)
svm_rbf = SVC(kernel='rbf', C = 1, random_state = 0)
svm_poly = SVC(kernel='poly', degree = 2, C = 1, random_state = 0)

svm_linear.fit(X_train, y_train)
svm_rbf.fit(X_train, y_train)
svm_poly.fit(X_train, y_train)

SVC(C=1, degree=2, kernel='poly', random_state=0)

In [7]:
# Навчання моделі Random Forest
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [8]:
# Прогнозування
y_pred_linear = svm_linear.predict(X_test)
y_pred_rbf = svm_rbf.predict(X_test)
y_pred_poly = svm_poly.predict(X_test)
y_pred_rf = model_rf.predict(X_test)

In [9]:
# Оцінка результатів
svm_linear_report = classification_report(y_test, y_pred_linear, output_dict=True, zero_division=1)
svm_rbf_report = classification_report(y_test, y_pred_rbf, output_dict=True, zero_division=1)
svm_poly_report = classification_report(y_test, y_pred_poly, output_dict=True, zero_division=1)
rf_report = classification_report(y_test, y_pred_rf, output_dict=True)

# print("Classification Report для Linear SVM (без часових ознак):")
# print(svm_linear_report)
# print("Classification Report для RBF SVM (без часових ознак):")
# print(svm_rbf_report)
# print("Classification Report для Polynomial SVM (без часових ознак):")
# print(svm_poly_report)

# print("Classification Report для Random Forest (без часових ознак):")
# print(rf_report)

# Створюємо DataFrame для кожного звіту
svm_linear_df = pd.DataFrame(svm_linear_report).transpose()
svm_rbf_df = pd.DataFrame(svm_rbf_report).transpose()
svm_poly_df = pd.DataFrame(svm_poly_report).transpose()
rf_df = pd.DataFrame(rf_report).transpose()

# Об'єднуємо всі звіти в одну таблицю
final_report = pd.concat([svm_linear_df['precision'], svm_linear_df['recall'], svm_linear_df['f1-score'], svm_linear_df['support'],
                          svm_rbf_df['precision'], svm_rbf_df['recall'], svm_rbf_df['f1-score'], svm_rbf_df['support'],
                          svm_poly_df['precision'], svm_poly_df['recall'], svm_poly_df['f1-score'], svm_poly_df['support'],
                          rf_df['precision'], rf_df['recall'], rf_df['f1-score'], rf_df['support']], axis=1)

# Назначаємо назви стовпців для кожного моделювання
final_report.columns = ['Linear SVM Precision', 'Linear SVM Recall', 'Linear SVM F1-Score', 'Linear SVM Support',
                        'RBF SVM Precision', 'RBF SVM Recall', 'RBF SVM F1-Score', 'RBF SVM Support',
                        'Poly SVM Precision', 'Poly SVM Recall', 'Poly SVM F1-Score', 'Poly SVM Support',
                        'Random Forest Precision', 'Random Forest Recall', 'Random Forest F1-Score', 'Random Forest Support']

# Вивести таблицю
print(final_report)

              Linear SVM Precision  Linear SVM Recall  Linear SVM F1-Score  \
idle                      0.805392           0.782034             0.793541   
running                   0.855299           0.852973             0.854135   
stairs                    1.000000           0.000000             0.000000   
walking                   0.769762           0.854811             0.810060   
accuracy                  0.820314           0.820314             0.820314   
macro avg                 0.857613           0.622455             0.614434   
weighted avg              0.826481           0.820314             0.809965   

              Linear SVM Support  RBF SVM Precision  RBF SVM Recall  \
idle                15585.000000           0.960108        0.983702   
running             51120.000000           0.928843        0.907257   
stairs               2475.000000           1.000000        0.003232   
walking             27750.000000           0.805171        0.900108   
accuracy            

---
# Висновки
---
- **Random Forest** — найкраща модель для цього завдання, оскільки вона має високу точність і відгук по всіх класах. Для всіх класів (включаючи "stairs" та "walking") модель досягла майже 100% точності та відгуку. Наприклад, для класу "stairs" — точність 1.0 та recall 0.978.
- **Linear SVM** і **RBF SVM** працюють досить добре, але мають проблеми з певними класами (зокрема, "stairs").
- **Poly SVM** виглядає трохи менш стабільною і має гірші результати, ніж інші моделі.
- Для класу **"stairs"** всі моделі мають проблеми з **recall**, тому може бути корисно переробити вибірку або вдосконалити алгоритми для кращої роботи з рідкісними класами.


---
# 2. Навчання моделей з часовими ознаками
---
Додамо часові ознаки до датасету. Для цього створимо функцію, яка розраховує додаткові характеристики на основі часових рядів.

In [17]:
def calculate_time_features(data):
    """Розрахунок часових ознак для кожної осі"""
    features = {}

    # Ознаки для кожної осі (X, Y, Z)
    for axis in ['X', 'Y', 'Z']:
        col = f'accelerometer_{axis}'

        # Статистичні ознаки
        features[f'max_{axis}'] = data[col].max()
        features[f'min_{axis}'] = data[col].min()
        features[f'mean_{axis}'] = data[col].mean()
        features[f'std_{axis}'] = data[col].std()
        features[f'iqr_{axis}'] = iqr(data[col])
        features[f'median_{axis}'] = np.median(data[col])

        # Часові ознаки
        features[f'abs_energy_{axis}'] = np.sum(data[col] ** 2)
        features[f'max_min_diff_{axis}'] = features[f'max_{axis}'] - features[f'min_{axis}']

    return features

# Створення нового датасету з часовими ознаками
all_features = []

for activity in data_set['activity'].unique():
    activity_data = data_set[data_set['activity'] == activity]
    for idx in range(len(activity_data)):
        sample = activity_data.iloc[idx]
        features = calculate_time_features(sample)
        features['activity'] = activity
        all_features.append(features)

enhanced_data = pd.DataFrame(all_features)


In [18]:
# Підготовка даних для навчання
X_enhanced = enhanced_data.drop('activity', axis=1)
y_enhanced = enhanced_data['activity']

In [19]:
X_enhanced.isna().sum()

max_X             0
min_X             0
mean_X            0
std_X             0
iqr_X             0
median_X          0
abs_energy_X      0
max_min_diff_X    0
max_Y             0
min_Y             0
mean_Y            0
std_Y             0
iqr_Y             0
median_Y          0
abs_energy_Y      0
max_min_diff_Y    0
max_Z             0
min_Z             0
mean_Z            0
std_Z             0
iqr_Z             0
median_Z          0
abs_energy_Z      0
max_min_diff_Z    0
dtype: int64

In [20]:
# Нормалізація даних
X_enhanced_normalized = scaler.fit_transform(X_enhanced)

In [21]:
# Розділення на тренувальний та тестовий набори
X_train_enhanced, X_test_enhanced, y_train_enhanced, y_test_enhanced = train_test_split(
    X_enhanced_normalized, y_enhanced, test_size=0.5, stratify=y_enhanced, random_state=42
)

In [22]:
# Навчання моделі SVM з часовими ознаками
model_svm_enhanced = SVC(kernel='linear')
model_svm_enhanced.fit(X_train_enhanced, y_train_enhanced)

# Навчання моделі Random Forest з часовими ознаками
model_rf_enhanced = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf_enhanced.fit(X_train_enhanced, y_train_enhanced)

# Прогнозування
y_pred_svm_enhanced = model_svm_enhanced.predict(X_test_enhanced)
y_pred_rf_enhanced = model_rf_enhanced.predict(X_test_enhanced)

# Оцінка результатів
svm_report_enhanced = classification_report(
    y_test_enhanced, y_pred_svm_enhanced, output_dict=True, zero_division=0
)

rf_report_enhanced = classification_report(
    y_test_enhanced, y_pred_rf_enhanced, output_dict=True, zero_division=0
)

In [23]:
# Оцінка результатів для SVM
svm_df_time = pd.DataFrame(svm_report_enhanced).T  # щоб зробити метрики в стовпцях

# Оцінка результатів для Random Forest=True)
rf_df_time = pd.DataFrame(rf_report_enhanced).T

# Виведення результатів у вигляді таблиць

print("Classification Report для SVM (без часових ознак):")
print(svm_rbf_df)

print("Classification Report для SVM (з часовими ознаками):")
print(svm_df_time)

print("\nClassification Report для Random Forest (з часовими ознаками):")
print(rf_df_time)

Classification Report для SVM (без часових ознак):
              precision    recall  f1-score       support
idle           0.960108  0.983702  0.971762  15585.000000
running        0.928843  0.907257  0.917923  51120.000000
stairs         1.000000  0.003232  0.006444   2475.000000
walking        0.805171  0.900108  0.849997  27750.000000
accuracy       0.894419  0.894419  0.894419      0.894419
macro avg      0.923530  0.698575  0.686531  96930.000000
weighted avg   0.900281  0.894419  0.883860  96930.000000
Classification Report для SVM (з часовими ознаками):
              precision    recall  f1-score       support
idle           0.947213  0.985563  0.966007  15585.000000
running        0.932344  0.883666  0.907353  51120.000000
stairs         0.000000  0.000000  0.000000   2475.000000
walking        0.775935  0.902126  0.834286  27750.000000
accuracy       0.882771  0.882771  0.882771      0.882771
macro avg      0.663873  0.692839  0.676911  96930.000000
weighted avg   0.866150  0

## **Підсумки:**

### **1. Порівняння SVM без і з часовими ознаками**
- **Точність (Precision) та F1-score** для класу **stairs** значно погіршилися при додаванні часових ознак (**0.000**), що вказує на проблеми з класифікацією цього класу.
- **Загальна точність (accuracy)** знизилася з **89.4% до 88.2%**, що свідчить про те, що додавання часових ознак не покращило модель.
- **Recall для класу walking** трохи зменшився, що може вказувати на те, що модель гірше розпізнає ці класи.

### **2. Random Forest з часовими ознаками проти SVM**
- **Random Forest значно перевищує SVM** за всіма метриками.
- **Для класу stairs** Recall = **0.978 (97.8%)**, у той час як у SVM він падає до **0.000** (тобто модель не може правильно класифікувати цей клас).
- **Загальна точність Random Forest** **99.93%**, що набагато краще, ніж у SVM (~88-89%).

## **Висновки:**
- **Додавання часових ознак не покращує роботу SVM**, а навпаки, може знижувати продуктивність.
- **Random Forest значно ефективніший за SVM**, особливо для складного класу **stairs**.

## **Рекомендації:**
- Якщо метою є **максимальна точність**, **Random Forest є найкращим вибором для цієї задачі**.


---
# 3. Порівняння Random Forest результатів
---
Тепер порівняємо результати обох підходів:

🚨 Чому Accuracy не завжди хороший показник?
Якщо в наборі даних 90% одного класу і лише 10% іншого, то модель може просто завжди передбачати більшість і отримати Accuracy = 90%, але реальна якість буде низькою.

Тому важливо дивитися ще на Precision, Recall та F1-score, особливо якщо дані незбалансовані як у нашому датасеті.

In [25]:
print("\nClassification Report для Random Forest (без часовх ознак):")
print(rf_df)
print("\nClassification Report для Random Forest (з часовими ознаками):")
print(rf_df_time)


Classification Report для Random Forest (без часовх ознак):
              precision    recall  f1-score       support
idle           0.999872  1.000000  0.999936  15585.000000
running        0.999120  0.999961  0.999540  51120.000000
stairs         1.000000  0.977778  0.988764   2475.000000
walking        0.999027  0.999387  0.999207  27750.000000
accuracy       0.999237  0.999237  0.999237      0.999237
macro avg      0.999505  0.994282  0.996862  96930.000000
weighted avg   0.999237  0.999237  0.999234  96930.000000

Classification Report для Random Forest (з часовими ознаками):
              precision    recall  f1-score       support
idle           0.999936  1.000000  0.999968  15585.000000
running        0.999101  0.999980  0.999540  51120.000000
stairs         1.000000  0.978586  0.989177   2475.000000
walking        0.999135  0.999387  0.999261  27750.000000
accuracy       0.999268  0.999268  0.999268      0.999268
macro avg      0.999543  0.994488  0.996987  96930.000000
weigh

## Висновки:
1. **Часові ознаки не змінюють суттєво ефективність моделі.** Заміна або додавання часових ознак не призвела до помітного поліпшення чи погіршення результатів.
2. **Модель Random Forest з часовими ознаками** все ще показує високу точність, відгук і f1-скор на всіх класах, зокрема на класі `stairs`, який раніше був складним.
3. **Не було помітного покращення з додаванням часових ознак,** і це свідчить про те, що або часові ознаки не мають суттєвого значення для задачі, або їх значення не було належним чином оброблено або інтегровано в модель.

## Рекомендації:
- Якщо покращення результатів з використанням часових ознак не видно, можна досліджувати **інші типи ознак** або спробувати змінити параметри моделі.
- **Random Forest** залишатиметься сильним вибором для цієї задачі, оскільки показує стабільно високу продуктивність навіть без часових ознак.
